In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import tensorflow_probability as tfp

import numpy as np
import matplotlib.pyplot as plt
%load_ext tensorboard
 

In [2]:
from utils import *
from datasets import *
from run import *
from run_experiment import *
from run_schedule import *
from model import *

In [4]:
dataset_dir = 'data/img_align_celeba'
log_dir = 'TrainLog01'
train_cf = test_train_config
g0_config = test_g_config
d0_config = test_d_config
train_cf.img_size= (16, 16)
train_cf.targ_img_size = (16, 16)
train_cf.applied_G_method = None
# train_cf.targ_img_size = (16, 16)
G_ = Generator_v0(img_size=train_cf.img_size, 
                targ_img_size=train_cf.targ_img_size, 
                z_dim=train_cf.latent_z_dim,
                apply_resize=train_cf.applied_G_method,
                 configbuild=g0_config)

D_ = Discriminator_v1(img_size=train_cf.img_size,
                    targ_img_size=train_cf.targ_img_size,
                    apply_resize=train_cf.applied_D_method,
                     configbuild=d0_config)

trainer = Trainer(G_, D_, 
                global_batch=50, 
                dataset_dir=dataset_dir,
                alpha_decay=True, 
                log_dir=log_dir,
                 TrainDict=train_cf)

trainer.initialize_trainer()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [5]:
import pickle

In [13]:
u = DefaultConfig(epochs=2, D_lr=1)

In [11]:
u.epochs

2